In [94]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import time



In [ ]:
# Put on GPU

device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")

In [95]:
# Extract images

noisy_imgs_1 , noisy_imgs_2 = torch.load('train_data.pkl')
noisy_imgs , clean_imgs = torch.load ('val_data.pkl')

noisy_imgs = noisy_imgs/255
clean_imgs = clean_imgs/255

print(noisy_imgs_1.size())

#plt.imshow(noisy_imgs[1].permute(1,2,0))
#plt.show()
#plt.imshow(clean_imgs[1].permute(1,2,0))
#plt.show()

#print(noisy_imgs_1[0,:,:,:])

#plt.imshow(noisy_imgs[0,1])
#plt.show()


# Utiliser ADAM en optimizer (0.01)


torch.Size([50000, 3, 32, 32])


In [96]:
# selct a preset of images:

imgs_1 = noisy_imgs_1[0:10000]/255
imgs_2 = noisy_imgs_2[0:10000]/255

In [97]:
class Net(nn.Module):
            def __init__(self):
                super().__init__()
               
                self.conv1 = nn.Conv2d(3,32,kernel_size=3,stride=1)
                self.conv2 = nn.Conv2d(32,32,kernel_size=5,stride=1)
                self.convT1 = nn.ConvTranspose2d(32,32,kernel_size=5,stride=1)
                self.convT2 = nn.ConvTranspose2d(32,3,kernel_size=3,stride=1)
                """
                self.conv1 = nn.Conv2d(3,32,kernel_size=2,stride=2)
                self.conv3 = nn.Conv2d(32,32,kernel_size=2,stride=2)
                self.convT1 = nn.ConvTranspose2d(32,32,kernel_size=2,stride=2)
                self.convT3 = nn.ConvTranspose2d(64,3,kernel_size=2,stride=2)
                """ 
            def forward(self, x):
                
                #print("1", x.shape)
                x = F.relu(self.conv1(x))
                #print("2", x.shape)
                x = F.max_pool2d(x, 3)
                #print("max_pool", x.shape)
                x = F.relu(self.conv2(x))
                #print("conv2", x.shape)
                x = F.relu(self.convT1(x))
                #print("conv_T_1", x.shape)
                x = F.upsample(x, size=None, scale_factor=3)
                #print("up_sample", x.shape)
                # use sigmoid
                x = F.sigmoid(self.convT2(x))
                #print("conv_T_2", x.shape)
                
                """
                print("1", x.shape)
                x = F.relu(self.conv1(x))
                print("conv_1", x.shape)
                x = F.max_pool2d(x, 3)
                print("pool_1", x.shape)
                x = F.upsample(x, size=None, scale_factor=2)
                print("upsample_1", x.shape)
                """
                """
                x1 = F.leaky_relu(self.conv1(x))
                x = F.leaky_relu(self.conv3(x1))
                x = F.leaky_relu(self.convT1(x))
                x = torch.cat((x,x1),1)
                x = F.leaky_relu(self.convT3(x))
                x = F.leaky_relu(x)
                """
                return x

In [102]:
### For mini - project 1



class Model () :
    def __init__( self ) -> None :
    ## instantiate model + optimizer + loss function + any other stuff you need
        
        self.device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")
        self.autoenc = Net().to(self.device)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.autoenc.parameters(), lr = 1e-2)
        
        pass

    def load_pretrained_model(self) -> None :
    ## This loads the parameters saved in bestmodel .pth into the model
        pass

    def train(self , train_input , train_target , mini_batch_size) -> None :
    #: train˙input : tensor of size (N, C, H, W) containing a noisy version of the images.

        
        train_input, train_target = train_input.to(self.device), train_target.to(self.device) # Use GPU
        
        nb_epochs = 40
        eta = 1e-1

        for e in range(nb_epochs):
            acc_loss = 0

            for b in range(0, train_input.size(0), mini_batch_size):
                output = self.autoenc(train_input.narrow(0, b, mini_batch_size))
                loss = self.criterion(output, train_target.narrow(0, b, mini_batch_size))
                acc_loss = acc_loss + loss.item()

                self.autoenc.zero_grad()
                loss.backward()
                self.optimizer.step()

                with torch.no_grad():
                    for p in self.autoenc.parameters():
                        p -= eta * p.grad

            print(e, acc_loss)

    #: train˙target : tensor of size (N, C, H, W) containing another noisy version of the same images , which only differs from the input by their noise .
        pass

    def predict(self , test_input ) -> torch.Tensor :
    #: test˙input : tensor of size (N1 , C, H, W) that has to be denoised by the trained or the loaded network .
    #: returns a tensor of the size (N1 , C, H, W)
        output = self.autoenc(test_input)
        
        return output
    
    # Add a method "load pretrained model" to load the best model


In [103]:
def psnr ( denoised , ground_truth ) :
        # Peak Signal to Noise Ratio : denoised and ground_truth have range [0 , 1]
        mse = torch.mean (( denoised - ground_truth ) ** 2)
        return -10 * torch.log10 ( mse + 10** -8)

In [104]:
imgs_1.shape[0]

10000

In [105]:
mini_batch_size = 1000

model = Model()
model.train( imgs_1, imgs_2, mini_batch_size)

out = model.predict(noisy_imgs)

img_denoise_psnr = psnr(out ,clean_imgs )


print("psnr", img_denoise_psnr)



/Users/antoine/opt/anaconda3/envs/deep_learning_env/lib/python3.9/site-packages/torch/nn/functional.py:3509: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/Users/antoine/opt/anaconda3/envs/deep_learning_env/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0 0.9249763339757919


KeyboardInterrupt: 

AttributeError: 'Model' object has no attribute 'state_dict'

In [ ]:
print(clean_imgs)

In [ ]:
# AFFICHAGE
print(out[1].shape)

#print(clean_imgs[1])
plt.imshow(clean_imgs[1].permute(1,2,0))
plt.show()
plt.imshow(out[1].permute(1,2,0).detach())
plt.show()
